In [1]:
import sys
import os
import subprocess
from typing import List, Tuple

# Ensure 'src' is in sys.path to allow direct Python imports (e.g., for `find_chars`)
sys.path.insert(0, os.path.abspath(os.path.join("..", "..", "src")))

import myproject.cli

In [2]:
import logging
from io import StringIO

log_stream = StringIO()
handler = logging.StreamHandler(log_stream)
logger = logging.getLogger("myproject.cli")
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

print("Logger configured and handler added.")

Logger configured and handler added.


In [3]:
logger.info("Processing query...")

print("Logged 'Processing query...'")
print("Current stream contents:")
print(log_stream.getvalue())

Logged 'Processing query...'
Current stream contents:
Processing query...



In [4]:
log_stream.truncate(0)
log_stream.seek(0)

print("Log stream cleared.")
print("Current stream contents after clearing:")
print(log_stream.getvalue())

Log stream cleared.
Current stream contents after clearing:



In [5]:
logger.info("Another message")

print("Logged 'Another message'")
print("Stream contents:")
print(log_stream.getvalue())

Logged 'Another message'
Stream contents:
Another message



In [6]:
logger.removeHandler(handler)

logger.info("Message after handler removed")

print("Logged message after removing handler")
print("Stream contents (should be unchanged):")
print(log_stream.getvalue())

Logged message after removing handler
Stream contents (should be unchanged):
Another message



In [4]:
import logging
from io import StringIO
from unittest import mock

# Setup StringIO stream and logging handler
log_stream = StringIO()
handler = logging.StreamHandler(log_stream)
logger = logging.getLogger("myproject.cli")
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

# Mock sys.argv for CLI args
mock_args = ["myproject", "--query", "logtest", "--color=never"]

with mock.patch.object(sys, "argv", mock_args):
    try:
        myproject.cli.main()
    except SystemExit as e:
        exit_code = e.code

# Read captured logs
logs = log_stream.getvalue()

print("Exit code:", exit_code)
print("Captured logs:\n", logs)

# Cleanup handler after test
logger.removeHandler(handler)

[RESULT]
Input query    : logtest
Processed value: dummy_value
Exit code: 0
Captured logs:
 [INFO] Processing query...

